# Presentacion: Analisis y prediccion del mercado de bitcoin

### Introducción
En la actualidad, el mercado de criptomonedas es un sector dinámico y fascinante que ofrece oportunidades únicas para el análisis de datos y la comprensión de las tendencias financieras. Este proyecto se enfoca en explorar este emocionante campo utilizando una de las herramientas más poderosas disponibles: la API de Binance.

### ¿Qué es Binance?
Binance es una de las plataformas de intercambio de criptomonedas más grandes y reconocidas a nivel mundial. Ofrece a sus usuarios una API rica y robusta que les permite acceder a una amplia gama de datos del mercado en tiempo real. Esto incluye datos históricos de precios, detalles de operaciones individuales, volúmenes de trading y mucho más.

### Objetivo del Proyecto
El objetivo principal de este proyecto es aprovechar los datos proporcionados por la API de Binance para obtener insights valiosos sobre el comportamiento del mercado de Bitcoin (BTC), la criptomoneda líder. Nos enfocaremos en extraer datos históricos de precios y operaciones de BTC en comparación con el USD, analizando las tendencias, volatilidades y posibles patrones de trading.

### Metodologia

- Descarga de Datos: Utilizaremos la API de Binance para descargar datos históricos de precios y operaciones de Bitcoin en intervalos de tiempo específicos.

- Limpieza y Preprocesamiento de Datos: Realizaremos una limpieza exhaustiva de los datos para eliminar valores atípicos, datos faltantes y cualquier otro factor que pueda afectar la calidad del análisis.

- Análisis Exploratorio de Datos (EDA): Exploraremos los datos para identificar tendencias, patrones y relaciones entre las variables. Esto nos ayudará a comprender mejor el comportamiento del mercado de Bitcoin.

- Modelado Predictivo: Utilizaremos técnicas de modelado predictivo para intentar prever el aumento o descenso de Bitcoin a corto plazo. Esto puede implicar el uso de modelos de regresión, series temporales u otras técnicas de aprendizaje automático.

- Evaluación de Modelos: Evaluaremos el rendimiento de nuestros modelos utilizando métricas relevantes y técnicas de validación cruzada para garantizar su robustez y precisión.

### Análisis de Datos del Mercado de Criptomonedas con Binance API

- En la era digital actual, el mercado de criptomonedas ha emergido como un campo fascinante y dinámico, ofreciendo oportunidades únicas para el análisis de datos y la comprensión de las tendencias del mercado financiero. Nuestro proyecto se centra en explorar este sector vibrante utilizando una de las herramientas más poderosas en el mundo de las criptomonedas: la API de Binance.

- Binance, siendo una de las plataformas de intercambio de criptomonedas más grandes y reconocidas a nivel mundial, proporciona una API rica y robusta, permitiendo a los usuarios acceder a una amplia gama de datos del mercado en tiempo real. Esto incluye, pero no se limita a, datos históricos de precios, detalles de operaciones individuales, volúmenes de trading, y mucho más.

- El objetivo principal de este proyecto es aprovechar estos datos para obtener insights valiosos sobre el comportamiento del mercado de Bitcoin (BTC), la criptomoneda líder en el mercado. Nos centraremos en extraer datos históricos de precios y operaciones de BTC comparados con el USD, analizando las tendencias, volatilidades y posibles patrones de trading.

- Al final de este proyecto, esperamos tener un análisis detallado y multifacético del mercado de Bitcoin, brindando un panorama claro no solo para los entusiastas de las criptomonedas, sino también para aquellos interesados en las finanzas y la tecnología. Este desafío es una excelente oportunidad para sumergirnos en el mundo del análisis de datos financieros y abrir la puerta a futuras investigaciones y proyectos en este apasionante campo.

# Descripcion

Para esto utilizaremos la API de binance a travez de su cliente en python, La cual nos deja acceso a muchas variables pero en estos momentos solo nos centraremos en el precio. 

Una vez el modelo este en funcionamiento, la misma api de binance permite realizar compras y ventas de BTC en un muy corto periodo de tiempo

In [1]:
# paquetes necesarios
import requests
import pandas as pd
import hmac
import hashlib
import time
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
from decouple import config
from binance.client import Client
import plotly.graph_objs as go
import statsmodels.api as sm
from pandas.plotting import register_matplotlib_converters
import statsmodels.api as sm
import ipywidgets as widgets
from IPython.display import display
api_key = config('API_KEY')
api_secret = config('API_SECRET')
api_limit = 1000
requestDays = 3365
limit = 1000

## Descargar ultimos precios

In [2]:
# Crear una instancia del cliente de Binance
client = Client(api_key, api_secret)

# Calcular las fechas para el último mes
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=requestDays)

# Convertir las fechas a milisegundos (que es el formato que la API de Binance espera)
start_date_timestamp = int(start_date.timestamp() * 1000)
end_date_timestamp = int(end_date.timestamp() * 1000)

# Obtener los datos del precio del Bitcoin para el último mes con intervalo de 1 minuto
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE

In [3]:

# Inicializar una lista para almacenar los datos
all_klines = []

# Hacer llamadas sucesivas para obtener más datos (limitando cada llamada a 500)

while start_date_timestamp < end_date_timestamp:
    klines = client.get_klines(symbol=symbol, interval=interval, startTime=start_date_timestamp, endTime=end_date_timestamp, limit=limit)
    
    # Verificar si hay datos en la respuesta
    if not klines:
        break
    
    all_klines.extend(klines)
    
    # Actualizar el startTime para la siguiente llamada
    start_date_timestamp = int((pd.to_datetime(klines[-1][0], unit='ms') + timedelta(minutes=1)).timestamp() * 1000)

# Comprobar si se recuperaron datos antes de crear el DataFrame
if all_klines:
    # Crear un DataFrame de Pandas con todos los datos
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    df = pd.DataFrame(all_klines, columns=columns)

    # Convertir el timestamp a formato de fecha y hora legible
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Mostrar el DataFrame
    df.info()
else:
    print("No se encontraron datos para el período especificado.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3449928 entries, 0 to 3449927
Data columns (total 12 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   timestamp                     datetime64[ns]
 1   open                          object        
 2   high                          object        
 3   low                           object        
 4   close                         object        
 5   volume                        object        
 6   close_time                    int64         
 7   quote_asset_volume            object        
 8   number_of_trades              int64         
 9   taker_buy_base_asset_volume   object        
 10  taker_buy_quote_asset_volume  object        
 11  ignore                        object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 315.9+ MB


In [4]:
df.sample(5)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
2181024,2021-10-15 16:56:00,61378.59000000,61426.22000000,61323.13000000,61361.18000000,98.15855000,1634317019999,6024730.29359200,2407,42.25078000,2593384.50346680,0
877989,2019-04-22 06:30:00,5232.00000000,5232.00000000,5227.23000000,5228.90000000,14.25812800,1555914659999,74579.91616074,145,7.11007800,37191.02703320,0
889195,2019-04-30 01:16:00,5245.44000000,5247.41000000,5245.43000000,5246.77000000,3.70547100,1556587019999,19439.31008679,89,2.00087900,10497.63477678,0
1214809,2019-12-13 03:34:00,7220.58000000,7222.16000000,7220.10000000,7222.00000000,25.61257800,1576208099999,184964.72910169,143,15.07088800,108838.49458055,0
1222307,2019-12-18 08:32:00,6641.43000000,6642.44000000,6635.75000000,6639.33000000,45.22073700,1576657979999,300280.41544959,269,23.79589300,158052.14339576,0


In [5]:
## save df to backup.json
df.to_json('backup.json')

In [2]:
## recuperar backup.json to df
df = pd.read_json('backup.json')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3449928 entries, 0 to 3449927
Data columns (total 12 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   timestamp                     datetime64[ns]
 1   open                          float64       
 2   high                          float64       
 3   low                           float64       
 4   close                         float64       
 5   volume                        float64       
 6   close_time                    datetime64[ns]
 7   quote_asset_volume            float64       
 8   number_of_trades              int64         
 9   taker_buy_base_asset_volume   float64       
 10  taker_buy_quote_asset_volume  float64       
 11  ignore                        int64         
dtypes: datetime64[ns](2), float64(8), int64(2)
memory usage: 342.2 MB


In [13]:
# Lista de opciones para el dropdown
opciones = ['Año', 'Mes', 'Dia']

# Crear el dropdown con un ancho personalizado para la descripción
dropdown = widgets.Dropdown(options=opciones, description='Selecciona un rango de tiempo:', style={'description_width': 'initial'})

# Función para imprimir solo el valor seleccionado
def imprimir_valor_seleccionado(change):
    #print("Seleccionaste:", change.new)
    ## si el valor seleccionado es Año
    if change.new == 'Dia':
        # Filtra el DataFrame para obtener solo los datos del último día
        ultimo_dia = df[df['timestamp'].dt.date == df['timestamp'].max().date()]

        # Crea la figura de Plotly
        fig = go.Figure()

        # Añade la línea al gráfico
        fig.add_trace(go.Scatter(x=ultimo_dia['timestamp'], y=ultimo_dia['close'], mode='lines+markers', name='Precio de cierre'))

        # Configura el diseño del gráfico
        fig.update_layout(title='Precio de cierre del último día',
                        xaxis_title='Timestamp',
                        yaxis_title='Precio de cierre')

        # Añade la funcionalidad de mostrar información al pasar el cursor
        fig.update_layout(hovermode='x unified')

        # Muestra el gráfico
        fig.show()

        # Filtra el DataFrame para obtener solo los datos del último día
        ultimo_dia = df[df['timestamp'].dt.date == df['timestamp'].max().date()]
        # Calcular el máximo y el mínimo del precio en el último mes
        precio_maximo = ultimo_dia['close'].max()
        precio_minimo = ultimo_dia['close'].min()

        # Verificar si el precio es ascendente o descendente
        precio_inicio = ultimo_dia.iloc[0]['close']
        precio_fin = ultimo_dia.iloc[-1]['close']

        if precio_fin > precio_inicio:
            tendencia = "ASCENDENTE"
        elif precio_fin < precio_inicio:
            tendencia = "DESCENDENTE"
        else:
            tendencia = "CONSTANTE"

        # Calcular la variación porcentual del precio en el último mes
        variacion_porcentual = ((precio_fin - precio_inicio) / precio_inicio) * 100

        print("El precio máximo en el último dia:", "$", precio_maximo)
        print("El precio mínimo en el último dia:", "$", precio_minimo)
        print("La tendencia del precio en el último mes es", tendencia)
        print("La variación porcentual del precio en el último mes es:", "{:.2f}%".format(variacion_porcentual))
    elif change.new == 'Mes':
        # Filtra el DataFrame para obtener solo los datos del último mes
        ultimo_mes = df[df['timestamp'] > df['timestamp'].max() - timedelta(days=30)]

        # Crea la figura de Plotly
        fig = go.Figure()

        # Añade la línea al gráfico
        fig.add_trace(go.Scatter(x=ultimo_mes['timestamp'], y=ultimo_mes['close'], mode='lines+markers', name='Precio de cierre'))

        # Configura el diseño del gráfico
        fig.update_layout(title='Precio de cierre del último mes',
                        xaxis_title='Timestamp',
                        yaxis_title='Precio de cierre')

        # Añade la funcionalidad de mostrar información al pasar el cursor
        fig.update_layout(hovermode='x unified')

        # Muestra el gráfico
        fig.show()
        
        # Filtra el DataFrame para obtener solo los datos del último mes
        ultimo_mes = df[df['timestamp'] >= df['timestamp'].max() - pd.DateOffset(months=1)]
        # Calcular el máximo y el mínimo del precio en el último mes
        precio_maximo = ultimo_mes['close'].max()
        precio_minimo = ultimo_mes['close'].min()

        # Verificar si el precio es ascendente o descendente
        precio_inicio = ultimo_mes.iloc[0]['close']
        precio_fin = ultimo_mes.iloc[-1]['close']

        if precio_fin > precio_inicio:
            tendencia = "ASCENDENTE"
        elif precio_fin < precio_inicio:
            tendencia = "DESCENDENTE"
        else:
            tendencia = "CONSTANTE"

        # Calcular la variación porcentual del precio en el último mes
        variacion_porcentual = ((precio_fin - precio_inicio) / precio_inicio) * 100

        print("El precio máximo en el último mes:", "$", precio_maximo)
        print("El precio mínimo en el último mes:", "$", precio_minimo)
        print("La tendencia del precio en el último mes es", tendencia)
        print("La variación porcentual del precio en el último mes es:", "{:.2f}%".format(variacion_porcentual))
    elif change.new == 'Año':
        # Filtra el DataFrame para obtener solo los datos del último año
        ultimo_anio = df[df['timestamp'] > df['timestamp'].max() - timedelta(days=365)]

        # Crea la figura de Plotly
        fig = go.Figure()

        # Añade la línea al gráfico
        fig.add_trace(go.Scatter(x=ultimo_anio['timestamp'], y=ultimo_anio['close'], mode='lines+markers', name='Precio de cierre'))

        # Configura el diseño del gráfico
        fig.update_layout(title='Precio de cierre del último año',
                        xaxis_title='Timestamp',
                        yaxis_title='Precio de cierre')

        # Añade la funcionalidad de mostrar información al pasar el cursor
        fig.update_layout(hovermode='x unified')

        # Muestra el gráfico
        fig.show()
        # Calcular el máximo y el mínimo del precio en el último año
        ultimo_anio = df[df['timestamp'] > df['timestamp'].max() - timedelta(days=365)]
        precio_maximo = ultimo_anio['close'].max()
        precio_minimo = ultimo_anio['close'].min()

        # Verificar si el precio es ascendente o descendente
        precio_inicio = ultimo_anio.iloc[0]['close']
        precio_fin = ultimo_anio.iloc[-1]['close']

        if precio_fin > precio_inicio:
            tendencia = "ASCENDENTE"
        elif precio_fin < precio_inicio:
            tendencia = "DESCENDENTE"
        else:
            tendencia = "CONSTANTE"

        # Calcular la variación porcentual del precio en el último mes
        variacion_porcentual = ((precio_fin - precio_inicio) / precio_inicio) * 100

        print("El precio máximo en el último mes:", "$", precio_maximo)
        print("El precio mínimo en el último mes:", "$", precio_minimo)
        print("La tendencia del precio en el último mes es", tendencia)
        print("La variación porcentual del precio en el último mes es:", "{:.2f}%".format(variacion_porcentual))

# Vincular la función con el evento de cambio de valor del dropdown
dropdown.observe(imprimir_valor_seleccionado, names='value')

# Mostrar el dropdown
display(dropdown)

Dropdown(description='Selecciona un rango de tiempo:', options=('Año', 'Mes', 'Dia'), style=DescriptionStyle(d…

El precio máximo en el último dia: $ 73775.54
El precio mínimo en el último dia: $ 68781.74
La tendencia del precio en el último mes es DESCENDENTE
La variación porcentual del precio en el último mes es: -2.05%


En este caso se denota una tendencia bajista del precio, Se puede correlacionar con el fin del pico de compra, Pero se espera que remonte dado el "Halving", el cual es un momento en donde la cantidad de bitcoins que se minan por bloque se divide

Con este grafico se puede ver que el precio del bitcoin se disparo el 26 de febrero en donde se acepto al bitcoin como una moneda en la bolsa de comercio de NY

In [5]:
### evaluar que variables pueden servir para entrenar un modelo
df.describe()


,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
count,3449928,3.449928e+06,3.449928e+06,3.449928e+06,3.449928e+06,3.449928e+06,3449928,3.449928e+06,3.449928e+06,3.449928e+06,3.449928e+06,3449928.0
mean,2020-12-01 13:56:42.856472064,2.180990e+04,2.182250e+04,2.179718e+04,2.180991e+04,5.043743e+01,2020-12-01 13:57:42.855323648,1.149345e+06,1.009825e+03,2.508999e+01,5.698324e+05,0.0
min,2017-08-17 04:00:00,2.830000e+03,2.830000e+03,2.817000e+03,2.817000e+03,0.000000e+00,2017-08-17 04:00:59.999000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,2019-04-11 12:02:45,8.060050e+03,8.066000e+03,8.054538e+03,8.060000e+03,1.075260e+01,2019-04-11 12:03:44.999000064,1.222412e+05,1.660000e+02,4.840861e+00,5.407468e+04,0.0
50%,2020-12-02 02:10:30,1.669450e+04,1.670060e+04,1.668800e+04,1.669462e+04,2.277845e+01,2020-12-02 02:11:29.999000064,3.961276e+05,4.400000e+02,1.096366e+01,1.851000e+05,0.0
75%,2022-07-24 22:37:15,3.363267e+04,3.366263e+04,3.360420e+04,3.363254e+04,5.088394e+01,2022-07-24 22:38:14.999000064,1.263727e+06,1.028000e+03,2.543893e+01,6.175883e+05,0.0
max,2024-03-14 22:39:00,7.377555e+04,7.377700e+04,7.368261e+04,7.377554e+04,5.877775e+03,2024-03-14 22:39:59.999000,1.459557e+08,1.073150e+05,3.537453e+03,8.947551e+07,0.0
std,NaN,1.654172e+04,1.655115e+04,1.653230e+04,1.654175e+04,9.438881e+01,NaN,2.303264e+06,1.773751e+03,4.807771e+01,1.181396e+06,0.0


# Establecer cuales variables son utiles para el modelo

In [6]:
df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
       'taker_buy_quote_asset_volume', 'ignore'],
      dtype='object')

In [7]:
# Definir el modelo
model_financial = 'close ~ open + high + low + volume + number_of_trades'

# Ajustar el modelo
lm_financial = sm.OLS.from_formula(model_financial, data=df).fit()

# Imprimir el resumen del modelo
lm_financial.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  close   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.145e+12
Date:                Thu, 14 Mar 2024   Prob (F-statistic):               0.00
Time:                        20:34:33   Log-Likelihood:            -1.3701e+07
No. Observations:             3449928   AIC:                         2.740e+07
Df Residuals:                 3449922   BIC:                         2.740e+07
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0242      0.012      1.996      0.046       0.000       0.048
open                -0.4663      0.000  -1175.871      0.000      -0.467      -0.466
high                 0.7345      0.000   2486.113      0.000       0.734       0.735
low                  0.7318      0.000   2480.184      0.000       0.731       0.732
volume               0.0004      0.000      2.637      0.008       0.000       0.001
number_of_trades  3.865e-05   8.71e-06      4.436      0.000    2.16e-05    5.57e-05
==============================================================================
Omnibus:                  2624414.870   Durbin-Watson:                   1.902
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      14971421674.877
Skew:                          -1.976   Prob(JB):                         0.00
Kurtosis:                     325.700   Cond. No.                     8.33e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.33e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""